In [1]:
from utils import build_url, save_json

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = 'RANKED_SOLO_5x5'

# Ligas de las que extraer jugadores
PIPELINE = [
    {
        'url': '/lol/league/v4/challengerleagues/by-queue/{queue}',
        'name': 'CHALLENGER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/grandmasterleagues/by-queue/{queue}',
        'name': 'GRANDMASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
        
    },
    {
        'url': '/lol/league/v4/masterleagues/by-queue/{queue}',
        'name': 'MASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'I'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'II',
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'III'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'IV'
        },
        'page': 1
    }
]

with open(API_KEY_FILE) as f:
    api_key = f.read()

In [2]:
import requests, copy, time
import os, shutil
import dill as pickle

start = time.time()

execution_cache_dir = 'current_execution'
leagues_state_file = f'{execution_cache_dir}/remaining_leagues.pkl'
players_file = f'{execution_cache_dir}/players.pkl'

headers = {
  'X-Riot-Token': api_key
}

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

# Leer un fichero del estado de la ejecución si existe o copiar PIPELINE
# para empezar desde 0
if os.path.exists(leagues_state_file) and os.stat(leagues_state_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        leagues_current_execution = pickle.load(f)
else:
    leagues_current_execution = copy.deepcopy(PIPELINE[0:3]) # Para pruebas

if os.path.exists(players_file) and os.stat(players_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        players = pickle.load(f)
else:
    players = []

# Filtrar las fases que ya estén completas
leagues_current_execution = list(filter(lambda league: not league['completed'], leagues_current_execution))

for league in leagues_current_execution:
    # Para las ligas paginadas, pedir en bucle hasta que no se devuelvan jugadores
    # Si alguna petición falla, aumentar delay
    # Si va bien, disminuir el delay hasta el inicial
    # Al terninar cada fase o página, guardar el estado
    if 'page' in league:
        params = league['params']
        while True:
            url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
            response = requests.get(url, headers=headers, params={'page': league['page']})
            if response.status_code == 200:
                players_temp = response.json()
                if not players_temp:
                    break
                players.extend(players_temp)
                print(f"{params['tier']} {params['division']} ({league['page']}): len: {len(players_temp)} / total: {len(players)}")
                delay = max(delay/2, MIN_DELAY)
                league['page'] += 1
            else:
                retry_after = int(response.headers['Retry-After'])
                delay += retry_after
                print(f"{params['tier']} {params['division']} ({league['page']}): Petición fallida: {response.text}")
                print(f'Esperando {retry_after}s')
            with open(leagues_state_file, 'wb') as f:
                pickle.dump(leagues_current_execution, f)
            with open(players_file, 'wb') as f:
                pickle.dump(players, f)
            time.sleep(delay)
    else:
        url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            players_temp = response.json()['entries']
            players.extend(players_temp)
            print(f"{league['name']}: len: {len(players_temp)} / total: {len(players)}")
            delay = max(delay/2, MIN_DELAY)
        else:
            retry_after = int(response.headers['Retry-After'])
            delay += retry_after
            print(f"{params['name']}: Petición fallida: {response.text}")
            print(f'Esperando {retry_after}s')
        time.sleep(delay)
    league['completed'] = True
    with open(leagues_state_file, 'wb') as f:
        pickle.dump(leagues_current_execution, f)
    with open(players_file, 'wb') as f:
        pickle.dump(players, f)
        
end = time.time()
print(f'\nEl proceso ha tardado {end - start}s')

CHALLENGER: len: 300 / total: 300
GRANDMASTER: len: 700 / total: 1000
MASTER: len: 5145 / total: 6145

El proceso ha tardado 3.012631893157959s


In [3]:
def get_account_with_retry(summoner: str, url: str, retries: int = 1) -> dict:
    response = requests.get(build_url(REGION, BASE_URL, url).format(summoner), headers=headers)
    if response.status_code == 200:
        account = response.json()
        return account
    else:
        retry_after = int(response.headers['Retry-After'])
        time.sleep(retry_after)
        account = get_account_with_retry(summoner, url, retries+1)
        return account


# Guardar añadir el accountId para poder pedir partidas
summoners = {player['summonerId'] for player in players[0:100]} # Para pruebas limitar

accounts_file = f'{execution_cache_dir}/accounts.pkl'
summoners_with_account_file = f'{execution_cache_dir}/summoners_with_account.pkl'

if os.path.exists(accounts_file) and os.stat(accounts_file).st_size != 0:
    with open(accounts_file, 'rb') as f:
        accounts = pickle.load(f)
else:
    accounts = []
    
if os.path.exists(summoners_with_account_file) and os.stat(summoners_with_account_file).st_size != 0:
    with open(summoners_with_account_file, 'rb') as f:
        summoners_with_account = pickle.load(f)
else:
    summoners_with_account = set()

url = '/lol/summoner/v4/summoners/{}'

summoners = summoners - summoners_with_account

# Para probar, solo coger los 100 primeros
for summoner in summoners:
    account = get_account_with_retry(summoner, url)
    print(account['accountId'])
    accounts.append({
        'summoner': summoner,
        'account': account['accountId']
    })
    summoners_with_account.add(summoner)
    with open(accounts_file, 'wb') as f:
        pickle.dump(accounts, f)
    with open(summoners_with_account_file, 'wb') as f:
        pickle.dump(summoners_with_account, f)
    time.sleep(delay)

save_json(accounts, 'accounts.json')

NHAwlcaEA7W0O9O22xJFjh4-cr87k_aMn9IhSO5--8Drpg
KUv-1jnybQMGnE10GJrhkkyipiuAfoSAWcUEXDx58ZCFAA
94I6mpP-kqkgnX0ky0XQ8yr_5WGMmuIBP-ys2BsJyexAGA
vdwXvZKVkgizv3A5_45e0ifSximuHC6mc410ZddOC0hmbjnOu6USLca2
4Y9sbDuO1x6TceeVZ_USeKhOKWBKPYpaKDhEg3o1yIcdXg
nxFXl5Shms5fh14pC1_jdn4QCarUyYloRwedUn4-JPlUblpyFwbZYpmO
ZWA12EjEwx-jSNLp9lX5L8JFLDgOVpq5p1kESEVhv6ipMQ
wPqBgDFihJqG12gh1b0Ii0rzNlIbz8LVoaXaDwSIDbCXRbBlIBwobk_7
ROsYQxqugIbvTfJGH8nFuMKK6hLRT89JchJRYTvExarPEfU
F0oHIKWMD5oBD5DdRPHrL7YEMIyAchV4R9V-8ROH8uiY8Q
UTZzhpTVbDJrnZgmlGbbnjtQmcqEDKDvOdtgDEbpvope_A
QlTUE3rXGqsdQ1GSQ7VwY4nXoRcaGGSJxw8UPGkmjw53Hg
YHYfHShJtVdi4PWDjXWN9GnsblK4WPoP9gMVVnbuWEO6bw
Yf6z93udtJgRHdsncrhCPboKtOyVl7iTTumtKbrui8q2b5CR8r-npNXF
NcEWxQSlGVMGh-hCtTESSYMjhuzEKO1FqQrkSH6hSqtpdw
bOi0-U1__gXY_DSgFzEOpoeY31pSnFv2T1okFlq0bFO8XtA
BXZtSlPfDGu8nI0PCj_Yc-3G4PXIUPHPJuXnpasmKPVVeNmgawI-Jc_x
LbphAymEJSOEtUGLDI2QoznjVZ1BTOr54WUPOAMds2j2Fg
NurK8I9qh41s-fNUH3kRWS4PD7AXTuYhcC5ZWJmQyTbY6H8rQkdJcUeM
YPMktbHHwViQ1cOlHLV9DrEBRk910JfLI-4TAqP5-X4P6

In [4]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)